# Лекция 5: Введение в нейронные сети

## Архитектура

**Нейрон** — базовая вычислительная единица нейронной сети. Каждый нейрон получает на вход вектор значений, умножает его на вектор весов, прибавляет смещение и пропускает результат через функцию активации. Математически это описывается формулой:
$$
a = \sigma\left(\sum_{i=1}^{n} w_i x_i + b\right),
$$
где:
- $x_i$ — входные данные,
- $w_i$ — соответствующие веса,
- $b$ — смещение (bias),
- $\sigma$ — функция активации (например, Sigmoid, ReLU, tanh).


**Структура нейронной сети** включает следующие компоненты:
- **Входной слой:** Принимает исходные данные и передаёт их на последующие слои.
- **Скрытые слои:** Один или более слоев, где происходит обработка информации. Количество и размер скрытых слоев определяют глубину сети. Слои бывают разных типов в зависимости от задачи.
- **Выходной слой:** Выдает окончательный результат модели (например, регрессионное значение или вероятности классов).

<img src="https://www.researchgate.net/profile/Sufyan-Al-Janabi/publication/335856901/figure/fig2/AS:804017303191552@1568704071230/a-Simple-neural-network-architecture-b-Simple-architecture-of-deep-neural-network.ppm" alt="Описание изображения" width="800">

### Входной слой (Input Layer)
- Принимает входные данные и передает их в сеть для дальнейшей обработки.
- Его основная задача преобразовать данные в форму, пригодную для обработки. Преобразует данные в признаковое пространство. Требует векторизованного (их называют эмбеддинги) или матричного представления данных.

In [ ]:
import torch


batch_size = 4
num_features = 10
input_tensor = torch.randn(batch_size, num_features)

print("Входной тензор:")
print(input_tensor)
print("Размер входного тензора:", input_tensor.shape)

### Полносвязный слой (Dense, Fully Connected Layer)
- Является базовым строительным блоком нейронной сети.
- Выполняет линейное преобразование входного вектора с последующей нелинейной активацией.
- Формула работы нейрона:
  $$
  a = \sigma(Wx + b),
  $$
  где:
  - $W$ — матрица весов,
  - $b$ — смещение (bias),
  - $\sigma$ — функция активации (ReLU, Sigmoid, tanh и т.д.).

In [ ]:
import torch
import torch.nn as nn


in_features = 10 # - размерность входного вектора
out_features = 20 # - размерность выходного вектора
bias = True # - использование смещения (по умолчанию True)

fc_layer = nn.Linear(in_features, out_features, bias)

batch_size = 32
x = torch.randn(batch_size, in_features)
output = fc_layer(x)

print("Размер входного тензора:", x.shape)
print("Размер выхода полносвязного слоя:", output.shape)

### Сверточный слой (Convolutional Layer)
- Применяется для обработки данных с пространственной структурой.
- Работает с локальными областями входных данных, используя фильтры (ядра свертки), которые перемещаются по изображению.
- Применяется в обработке изображений и видео, сигналов и временных рядов.

Параметры:
- *Размер ядра (Kernel Size):* Размер фильтра.
- *Шаг (Stride):* Количество пикселей, на которое перемещается фильтр.
- *Отступ (Padding):* Добавление дополнительных значений по краям для сохранения размерности.
- *Количество каналов (Depth):* Количество применяемых фильтров.

<img src="https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif" alt="Описание изображения" width="600">

In [ ]:
!pip install torchvision -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import datasets, transforms


transform = transforms.Compose([transforms.ToTensor()])
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
image, label = dataset[0]

in_channels = 1 # - число входных каналов
out_channels = 8 # - число фильтров (выходных каналов)
kernel_size = 3 # - размер ядра свертки
stride = 1 # - шаг свертки (по умолчанию 1)
padding = 1 # - заполнение краев (для сохранения размерности)

conv_layer = nn.Conv2d(in_channels, out_channels, 
                       kernel_size, stride, padding)

input_tensor = image.unsqueeze(0)
conv_output = F.relu(conv_layer(input_tensor))

print("Размер входного тензора:", input_tensor.shape)
print("Размер выхода сверточного слоя:", conv_output.shape)

plt.figure(figsize=(6, 4))

plt.subplot(1, 2, 1)
plt.title("Входное изображение")
plt.imshow(image.permute(1, 2, 0))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Выход первого свертки")
output_feature = conv_output[0, 0].detach().numpy()
plt.imshow(output_feature, cmap='gray')
plt.axis('off')
plt.tight_layout()
plt.show()

### Слои пулинга (Pooling Layers)
- Сокращают пространственные размеры данных, помогая уменьшить вычислительную сложность и предотвращая переобучение.
- Неразрывно связан со сверточными слоями, позволяя архитектуре в целом видеть паттерны на изображении, выделяет наиболее существенные признаки.

Варианты:
- *Max Pooling:* Выбирает максимум из выборки.
- *Average Pooling:* Вычисляет среднее значение.

<img src="https://nico-curti.github.io/NumPyNet/NumPyNet/images/maxpool.gif" alt="Описание изображения" width="600">

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


transform = transforms.Compose([transforms.ToTensor()])
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
image, label = dataset[0]

kernel_size = 3 # - размер окна пулинга
stride = 3 # - шаг пулинга (по умолчанию равен kernel_size)
padding = 0 # - заполнение (обычно 0)
pool_layer = nn.MaxPool2d(kernel_size, stride, padding)

input_tensor = image.unsqueeze(0)
pooled_output = pool_layer(input_tensor)

print("Входное изображение shape:", input_tensor.shape)
print("После пулинга shape:", pooled_output.shape)

plt.figure(figsize=(6, 4))

plt.subplot(1, 2, 1)
plt.title("Входное изображение")
plt.imshow(image.permute(1, 2, 0))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Изображение после пулинга")
pooled_img = pooled_output[0, 0].detach().numpy()
plt.imshow(pooled_img, cmap='viridis')
plt.axis('off')

plt.tight_layout()
plt.show()

### Рекуррентный слой (Recurrent Layer)
- Применяется для работы с последовательными данными, учитывая порядок элементов.

Варианты:
- *RNN (Recurrent Neural Network):* Простая структура, но подверженная проблемам исчезающего и "взрывающегося" градиента.
- *LSTM (Long Short-Term Memory):* Имеет механизмы запоминания и забывания, позволяет работать с длительными зависимостями.
- *GRU (Gated Recurrent Units):* Более простой вариант, чем LSTM, но с подобной функциональностью.
- *State space models (Mamba)*

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20231204130132/RNN-vs-FNN-660.png" alt="Описание изображения" width="600">

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


class LSTMSequenceModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1):
        super(LSTMSequenceModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, # - размерность входного вектора
                            hidden_dim, # - размерность скрытого состояния
                            num_layers, # - число слоев LSTM
                            batch_first=True # - если True, первый размер соответствует batch_size
                            )
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embed = self.embedding(x)
        out, (h_n, _) = self.lstm(embed)
        last_hidden = h_n[-1]
        logits = self.fc(last_hidden)
        return logits


def create_dataset(sequence, seq_len):
    """
    Создает датасет для предсказания следующего токена.
    Для последовательности [t0, t1, t2, ...] формируются примеры:
    вход: [t0, t1, ..., t(seq_len-1)], 
    цель: t(seq_len)
    """
    inputs, targets = [], []
    for i in range(len(sequence) - seq_len):
        inputs.append(sequence[i:i + seq_len])
        targets.append(sequence[i + seq_len])
    return torch.tensor(inputs, dtype=torch.long), torch.tensor(targets, dtype=torch.long)


vocab = [0, 1, 2]
vocab_size = len(vocab)
embed_dim = 8
hidden_dim = 16
seq_len = 5
num_epochs = 100
learning_rate = 0.01

base_pattern = [0, 1, 2]
sequence = base_pattern * 50

inputs, targets = create_dataset(sequence, seq_len)

model = LSTMSequenceModel(vocab_size, embed_dim, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

losses = []
for epoch in tqdm(range(num_epochs)):
    model.train()
    optimizer.zero_grad()
    logits = model(inputs)
    loss = criterion(logits, targets)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

plt.figure(figsize=(4, 4))
plt.plot(losses, label='Потери')
plt.xlabel('Эпоха')
plt.ylabel('Loss')
plt.title('Обучение LSTM для предсказания следующего элемента')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

model.eval()
test_seq = sequence[-seq_len:]
test_input = torch.tensor([test_seq], dtype=torch.long)
with torch.no_grad():
    logits = model(test_input)
    probabilities = torch.softmax(logits, dim=1)
    predicted_token = torch.argmax(probabilities, dim=1).item()

print("Тестовая последовательность:", test_seq)
print("Предсказанный следующий элемент:", predicted_token)
print("Вероятности:", np.round(probabilities[0].numpy(), 3))

### Слои внимания (Attention Layers)
- Позволяют модели фокусироваться на наиболее значимых частях входных данных.
  
Варианты:
- *Cross-Attention:* 
- *Self-Attention:* Используется для обработки последовательностей, где каждый элемент взаимодействует с другими.
- *Multi-Head Attention:* Позволяет модели учитывать информацию с разных "углов зрения", что особенно важно в трансформерах.

<img src="https://jalammar.github.io/images/t/self-attention-output.png" alt="Описание изображения" width="600">

<img src="https://jalammar.github.io/images/t/transformer_decoding_2.gif" alt="Описание изображения" width="600">

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

class AttentionSequenceModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, max_seq_len, hidden_dim):
        super(AttentionSequenceModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq_len, embed_dim)
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        self.fc = nn.Linear(embed_dim, vocab_size)
        self.max_seq_len = max_seq_len

    def forward(self, x):
        batch_size, seq_len = x.size()
        token_emb = self.embedding(x)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.pos_embedding(positions)
        x_emb = token_emb + pos_emb

        attn_mask = torch.triu(torch.ones(seq_len, seq_len, device=x.device)
                               * float('-inf'), diagonal=1)
        attn_output, _ = self.attention(x_emb, x_emb, x_emb, attn_mask=attn_mask)
        last_output = attn_output[:, -1, :]
        logits = self.fc(last_output)
        return logits


def create_dataset(sequence, seq_len):
    """
    Создаем датасет для предсказания следующего токена (аналогично LSTM-примера).
    """
    inputs, targets = [], []
    for i in range(len(sequence) - seq_len):
        inputs.append(sequence[i:i + seq_len])
        targets.append(sequence[i + seq_len])
    return torch.tensor(inputs, dtype=torch.long), torch.tensor(targets, dtype=torch.long)


vocab = [0, 1, 2]
vocab_size = len(vocab)
embed_dim = 8
num_heads = 2
max_seq_len = 10
hidden_dim = 16
seq_len = 5
num_epochs = 100
learning_rate = 0.01

base_pattern = [0, 1, 2]
sequence = base_pattern * 50

inputs, targets = create_dataset(sequence, seq_len)

model = AttentionSequenceModel(vocab_size, embed_dim, num_heads, max_seq_len, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

losses = []
for epoch in tqdm(range(num_epochs)):
    model.train()
    optimizer.zero_grad()
    logits = model(inputs)
    loss = criterion(logits, targets)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

plt.figure(figsize=(4, 4))
plt.plot(losses, label='Потери')
plt.xlabel("Эпоха")
plt.ylabel("Loss")
plt.title("Обучение модели внимания для предсказания следующего элемента")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

model.eval()
test_seq = sequence[-seq_len:]
test_input = torch.tensor([test_seq], dtype=torch.long)
with torch.no_grad():
    logits = model(test_input)  # -> [1, vocab_size]
    probabilities = torch.softmax(logits, dim=1)
    predicted_token = torch.argmax(probabilities, dim=1).item()

print("Тестовая последовательность:", test_seq)
print("Предсказанный следующий элемент:", predicted_token)
print("Вероятности:", np.round(probabilities[0].numpy(), 3))

### Слой нормализации (Normalization Layer)
- Повышает стабильность и скорость обучения, нормализуя активации внутри слоев.
- Снижает проблемы исчезающего или "взрывающегося" градиента.
  
Варианты:
- *Batch Normalization:* Нормализация по батчам.
- *Layer Normalization:* Нормализация по признакам отдельного образца.
- *Instance Normalization и Group Normalization:* Используются в специфичных архитектурах (например, в задачах стилизации изображений).

<img src="https://theaisummer.com/static/ac89fbcf1c115f07ae68af695c28c4a0/ee604/normalization.png" alt="Описание изображения" width="600">

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


transform = transforms.Compose([transforms.ToTensor()])
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
image, label = dataset[0]

num_features = 1 # - число входных каналов
bn_layer = nn.BatchNorm2d(num_features)

input_tensor = image.unsqueeze(0)
mean_before = input_tensor.mean(dim=[0, 2, 3])
std_before = input_tensor.std(dim=[0, 2, 3])

normalized_image = bn_layer(input_tensor)

mean_after = normalized_image.mean(dim=[0, 2, 3])
std_after = normalized_image.std(dim=[0, 2, 3])

print("Статистика до нормализации:")
print(f"Mean: {float(mean_before):.3f}")
print(f"Std: {float(std_before):.3f}")
print("Статистика после нормализации:")
print(f"Mean: {float(mean_after):.3f}")
print(f"Std: {float(std_after):.3f}")

plt.figure(figsize=(6, 4))

plt.subplot(1, 2, 1)
plt.title("Входное изображение")
plt.imshow(image.permute(1, 2, 0))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Изображение после нормализации")
normalized_img = normalized_image[0, 0].detach().numpy()
plt.imshow(normalized_img, cmap='viridis')
plt.axis('off')

plt.tight_layout()
plt.show()

### Слой регуляризации
- Применяется для уменьшения переобучения модели за счет внесения случайности или ограничения значений параметров.

Варианты:
- *Dropout:* Случайное обнуление части нейронов, выбранных случайно во время обучения, что позволяет модели не слишком фокусироваться на связях между конкретными нейронами.
- [не слой, но все же] *L1 / L2 регуляризация:* Штрафование величины весов в функции потерь. Аналогично тому, как это было в линейных моделях.


<img src="https://editor.analyticsvidhya.com/uploads/112801.gif" alt="Описание изображения" width="600">

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


p = 0.5 # - вероятность зануления элемента входного тензора
dropout_layer = nn.Dropout(p)

input_tensor = torch.ones(100)

dropout_layer.train()
output = dropout_layer(input_tensor)

num_zeros = (output == 0).sum().item()
total_elements = output.numel()
fraction_dropped = num_zeros / total_elements

print(f"Доля зануленных элементов: {fraction_dropped:.2f}")

### Слои соединений Residual Connections

- Позволяют прямую передачу информации от одного слоя к другому, пропуская несколько промежуточных слоев.
- Способствуют лучшей передаче градиентов в глубоких сетях, решая проблему исчезающего градиента.

Применение:
- *ResNet:* Глубокие сверточные сети для задач классификации изображений.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/ResBlock.png/1200px-ResBlock.png" alt="Описание изображения" width="600">

In [ ]:
import json
import requests
from io import BytesIO

import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image


model = models.resnet18(pretrained=True)
model.eval()

class_index_url = (
    "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
)
response = requests.get(class_index_url)
json_str = response.content.decode("utf-8-sig")
class_idx = json.loads(json_str)
idx2label = {int(key): value[1] for key, value in class_idx.items()}

image_url = (
    "https://upload.wikimedia.org/wikipedia/commons/2/26/YellowLabradorLooking_new.jpg"
)
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert("RGB")

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)

with torch.no_grad():
    output = model(input_batch)

probabilities = F.softmax(output[0], dim=0)

top5_prob, top5_catid = torch.topk(probabilities, 3)

plt.figure(figsize=(4, 4))
plt.imshow(image)
plt.axis('off')
plt.show()

print("Результаты предсказания:")
for i in range(top5_prob.size(0)):
    print(f"{idx2label[top5_catid[i].item()]}: {top5_prob[i].item() * 100:.2f}%")

### Выходной слой (Output Layer)
- Формирует окончательное предсказание, преобразуя признаки, полученные из предыдущих слоев.

Варианты:
- Для *задач классификации:* 
  - Слой с функцией Softmax для многоклассовой классификации.
  - Слой с функцией Sigmoid для двоичной классификации.
- Для *задач регрессии:*
  - Линейный слой, зачастую без дополнительной функции активации или с подходящей активацией в зависимости от специфики задачи.

<img src="https://images.contentstack.io/v3/assets/bltac01ee6daa3a1e14/blte5e1674e3883fab3/65ef8ba4039fdd4df8335b7c/img_blog_image1_inline_(2).png?width=1024&disable=upscale&auto=webp" alt="Описание изображения" width="600">

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np


batch_size = 5
in_features = 10 # - размер входного вектора
out_features = 2 # - число классов
linear_layer = nn.Linear(in_features, out_features)
softmax = nn.Softmax(dim=1)

x = torch.randn(batch_size, in_features)
logits = linear_layer(x)
probabilities = softmax(logits)

print("Logits:\n", logits.detach().numpy())
print("Probabilities:\n", probabilities.detach().numpy())

plt.figure(figsize=(6, 4))
plt.bar(np.arange(out_features), probabilities[0].detach().numpy(), alpha=0.7)
plt.xlabel("Класс")
plt.ylabel("Вероятность")
plt.title("Распределение вероятностей для первого примера (Softmax Output)")
plt.grid(True)
plt.tight_layout()
plt.show()

## Обучение

### Гиперпараметры

**Гиперпараметры** — это параметры, которые задаются до начала обучения и существенно влияют на производительность модели:

- *Архитектура сети:*
  - Количество скрытых слоев и нейронов в каждом слое.  
  - Структура сети, используемые слои.
  - Функции активации и, если есть, их параметры.

- *Оптимизатор и скорость обучения (Learning Rate):* 
  Слишком высокий learning rate может привести к нестабильному обучению, а слишком низкий — к медленной сходимости. Может использоваться динамический шаг градиентного спуска, расписание, ранняя остановка.

- *Размер батча (Batch Size):*
  Количество примеров, используемых для одного обновления весов. Больший batch size может дать более стабильную оценку градиента, но требует больше оперативной памяти.

- *Количество эпох (Epochs):*
  Число полных проходов по обучающему набору данных. В общем случае эпох должно быть достаточно для установления постоянства по функции потерь.

- *Методы регуляризации:*
  Такие как L1/L2 (коэффициенты) регуляризация и dropout (вероятность), помогают уменьшить переобучение, сохраняя обобщающую способность модели.

Подбор этих гиперпараметров зачастую требует экспериментов и может быть оптимизирован с помощью методов поиска и концепции кросс-валидации (Grid Search, Random Search, Bayesian Optimization).

<img src="https://towardsdatascience.com/wp-content/uploads/2022/09/1_9coeJ4OSPqAjiKLcQqiew.png" alt="Описание изображения" width="600">

In [82]:
!pip install optuna -q

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split


class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_units, dropout_rate, num_classes):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_units)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_units, num_classes)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


def objective(trial):
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    hidden_units = trial.suggest_int("hidden_units", 64, 256)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    dataset = torchvision.datasets.MNIST(
        root="./data", train=True, download=True, transform=transform
    )
    
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

    model = SimpleMLP(input_size=28 * 28, hidden_units=hidden_units,
                      dropout_rate=dropout_rate, num_classes=10)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    epochs = 3
    for epoch in range(epochs):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                val_loss += loss.item() * data.size(0)
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total += data.size(0)
        val_loss /= total
        accuracy = correct / total

        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_loss


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

print("Лучший результат:")
best_trial = study.best_trial
print(f"Validation Loss: {best_trial.value:.4f}")
print("Лучшие гиперпараметры:")
for key, value in best_trial.params.items():
    print(f"{key}: {round(value, 3)}")

### Обучаемые параметры

**Обучаемые параметры** — это те числовые значения в модели, которые обновляются в процессе обучения с целью минимизации функции потерь (ошибки). В нейронных сетях к таким параметрам относятся:
- **Веса (Weights):** Коэффициенты, умножаемые на входные данные.
- **Смещения (Biases):** Константные слагаемые, добавляемые к результату линейного преобразования.

Описанные выше типы слоев позволяют решать различные типы задач, за счет оптимизации этих параметров. Это как бы дает "степени свободы" для модели.

Пример:
Для одного нейрона с входными данными $ x $, весами $ \mathbf{w} $ и смещением $ b $ предсказание вычисляется по формуле:
$$
a = \sigma\left(\langle \mathbf{w}, x \rangle + b\right),
$$
где $ \sigma $ — функция активации. 
В процессе обучения цель заключается в нахождении таких значений $ \mathbf{w} $ и $ b $, при которых функция потерь $ \mathcal{L} $ (например, среднеквадратичная ошибка) минимальна:
$$
\min_{\mathbf{w},\, b} \; \mathcal{L}(y, f(x; \mathbf{w}, b)).
$$

Для обучаемых параметров есть два основных этапа:
- *Инициализация*
- *Обновление*

#### Виды инициализации параметров
Правильная инициализация параметров — важный аспект, влияющий на скорость сходимости и качество обучения. В статьях, презентующих новые виды слове или функций активации ему обычно посвящают теоретический блок. Однако существуют общие подходы:

**Инициализация случайными значениями**
- *Uniform Initialization:* Весам присваиваются значения, равномерно распределённые в некотором интервале, например, $[-a, a]$.
- *Normal Initialization:* Значения весов берутся из нормального распределения с нулевым средним и заранее заданной дисперсией.

**Инициализация с учетом размеров слоев**
- *Xavier Initialization:* 
  - Разработана для эффективного распространения сигналов в сети с симметричными функциями активации (например, tanh, sigmoid). 
  - Весам присваиваются значения из распределения с дисперсией, зависящей от количества входов и выходов:
  $$
  \text{Var}(w) = \frac{2}{n_{in} + n_{out}}.
  $$
  - Xavier Uniform Initialization:
    $$
    W \sim U\left(-\sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}},\; \sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}\right).
    $$
  - Xavier Normal Initialization:
    $$
    W \sim \mathcal{N}\left(0,\; \sqrt{\frac{2}{n_{\text{in}} + n_{\text{out}}}}\right).
    $$
- *Kaiming Initialization:*
  - Адаптирована для слоев с функцией активации ReLU.
  - Весам присваиваются значения с дисперсией: $ \text{Var}(w) = \frac{2}{n_{in}} $, что помогает избежать проблем с затуханием или "взрывом" градиентов.

<img src="https://www.doc.ic.ac.uk/~nuric/posts/teaching/imperial-college-machine-learning-neural-networks/cover.gif" alt="Описание изображения" width="600">

[TensorFlow Playground](https://playground.tensorflow.org)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

class SimpleMLP(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=256, num_classes=10):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


def initialize_weights(model, init_type):
    """
    Функция инициализации весов для модели.
    
    Аргументы:
      model (torch.nn.Module): модель для инициализации весов.
      init_type (str): тип инициализации. Возможные варианты:
        - "xavier_uniform"
        - "kaiming_normal"
    """
    for m in model.modules():
        if isinstance(m, nn.Linear):
            if init_type == "xavier_uniform":
                init.xavier_uniform_(m.weight)
            elif init_type == "kaiming_normal":
                init.kaiming_normal_(m.weight, nonlinearity="relu")
            else:
                raise ValueError("Неизвестный тип инициализации")
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)


def train_model(model, train_loader, optimizer, criterion, device, num_epochs=5):
    """
    Функция обучения модели.
    """
    model.train()
    epoch_losses = []
    for epoch in tqdm(range(num_epochs)):
        running_loss = 0.0
        total = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * data.size(0)
            total += data.size(0)
        epoch_loss = running_loss / total
        epoch_losses.append(epoch_loss)
    return epoch_losses


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
subset_indices = np.arange(1000)
dataset = Subset(dataset, subset_indices)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True)

num_epochs = 10

model_xavier = SimpleMLP().to(device)
initialize_weights(model_xavier, init_type="xavier_uniform")
optimizer_xavier = optim.Adam(model_xavier.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
print("Обучение модели с инициализацией Xavier Uniform:")
losses_xavier = train_model(model_xavier, train_loader, optimizer_xavier, criterion, device, num_epochs)

model_kaiming = SimpleMLP().to(device)
initialize_weights(model_kaiming, init_type="kaiming_normal")
optimizer_kaiming = optim.Adam(model_kaiming.parameters(), lr=1e-3)
print("Обучение модели с инициализацией Kaiming Normal:")
losses_kaiming = train_model(model_kaiming, train_loader, optimizer_kaiming, criterion, device, num_epochs)

epochs = np.arange(1, num_epochs+1)
plt.figure(figsize=(8, 6))
plt.plot(epochs, losses_xavier, "o-", label="Xavier Uniform")
plt.plot(epochs, losses_kaiming, "s-", label="Kaiming Normal")
plt.xlabel("Эпоха")
plt.ylabel("Средняя Loss")
plt.title("Сравнение сходимости модели с разной инициализацией весов")
plt.legend()
plt.yscale('log')
plt.grid(True)
plt.tight_layout()
plt.show()

### Автоматическое дифференцирование

**Автоматическое дифференцирование** — это метод вычисления производных (градиентов) функции, с использованием алгоритмов, которые реализуют правило цепочки (производная сложной функции, chain rule).

Ключевые моменты:
- *Правило цепочки:* Автодифференцирование "распространяет" вычисления производных через весь вычислительный граф, начиная с выходного слоя и двигаясь к входным данным (обратный режим), или наоборот (прямой режим).
- *Виды:*
  - **Прямой режим (Forward Mode):** Эффективен, когда число входов невелико по сравнению с числом выходов.
  - **Обратный режим (Reverse Mode):** Чаще используется в нейронных сетях, так как число выходов, как правило, мало, а число параметров велико. Именно обратное распространение ошибки (backpropagation) является применением автодифференцирования.
- *Преимущества:*
  - Производные вычисляются с точностью машинного представления.
  - Вычисление градиента происходит за время, сравнимое с временем расчета функции, независимо от размерности входов.
  - Автоматизация вычислений.

  Метод не подвержен неусточивости, накоплению ошибок округления и неэффективному представлению в памяти в сравнении с конечными разностями или символьным дифференцированием.

<img src="https://alexander-schiendorfer.github.io/images/backprop/nnet-vanilla.gif" alt="Описание изображения" width="600">

In [ ]:
"""
Пример: Сравнение автодифференцирования и аналитического вычисления производной

В этом примере:
  - Определяется функция f(x) = x^3 + 2*x^2.
  - Вычисляется аналитическая производная f'(x) = 3*x^2 + 4*x.
  - С помощью механизма автодифференцирования в PyTorch вычисляется
    градиент этой же функции.
  - Результаты сравниваются на множестве точек и отображаются на графике.
"""

import torch
import matplotlib.pyplot as plt
import numpy as np

def f(x):
    return x**3 + 2*x**2

def analytic_derivative(x):
    return 3*x**2 + 4*x

xs = torch.linspace(-5, 5, 100, requires_grad=True)
ys = f(xs)

gradients = torch.autograd.grad(ys, xs, grad_outputs=torch.ones_like(ys), create_graph=True)[0]

plt.figure(figsize=(6, 6))

xs_np = xs.detach().numpy()
gradients_np = gradients.detach().numpy()
plt.plot(xs_np, gradients_np, label="Производная (Autograd)")

analytic_np = analytic_derivative(xs_np)
plt.plot(xs_np, analytic_np, label="Аналитическая производная", linestyle="--")

plt.xlabel("x")
plt.ylabel("Производная")
plt.title("Сравнение автодифференцирования и аналитического вычисления производной")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Библиотеки для нейронных сетей

Существуют несколько ключевых библиотек для разработки нейронных сетей на Python, каждая из которых имеет свои особенности и преимущества.

### TensorFlow
**TensorFlow** – это фреймворк, разработанный Google, который изначально основан на концепции статических вычислительных графов. Начиная с версии 2.x, активно используется режим "eager execution" - это режим, при котором операции выполняются немедленно при вызове, а не строятся в виде статического графа вычислений.

Особенности:
- Поддерживает как низкоуровневое программирование с построением вычислительных графов, так и высокоуровневое API через Keras - изначальной другой библиотеки, которая теперь входит в состав TF.
- Позволяет работать как на CPU, так и на GPU и TPU, что способствует ускорению обучения для больших моделей.
- Имеет TensorBoard для визуализации, TensorFlow Serving для развертывания.
- Надежный инструмент, качественно поддерживаемый Google.

In [ ]:
!pip install tensorflow numpy==1.26.0 -q

In [ ]:
import tensorflow as tf


class SimpleTFModel(tf.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, name=None):
        super(SimpleTFModel, self).__init__(name=name)
        self.w1 = tf.Variable(tf.random.normal([input_dim, hidden_dim]), name="w1")
        self.b1 = tf.Variable(tf.zeros([hidden_dim]), name="b1")
        self.w2 = tf.Variable(tf.random.normal([hidden_dim, output_dim]), name="w2")
        self.b2 = tf.Variable(tf.zeros([output_dim]), name="b2")

    def __call__(self, x):
        hidden = tf.nn.relu(tf.matmul(x, self.w1) + self.b1)
        output = tf.matmul(hidden, self.w2) + self.b2
        return output

input_dim = 5
hidden_dim = 10
output_dim = 2

model = SimpleTFModel(input_dim, hidden_dim, output_dim)
x = tf.random.normal([3, input_dim])
y = model(x)
print("Выход модели TensorFlow:")
print(y)

### Keras
**Keras** – это высокоуровневый API для построения нейронных сетей. Сейчас Keras интегрирован в TensorFlow и является его стандартным интерфейсом для быстрой сборки моделей.

Особенности:
- Обладает интуитивно понятным синтаксисом.
- Несмотря на высокоуровневый подход, позволяет достаточно подробно контролировать параметры модели.
- Богатая документация и обширное сообщество.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential


def build_keras_model(input_dim, hidden_dim, output_dim):
    model = Sequential([
        layers.Dense(hidden_dim, activation='relu', 
                     input_shape=(input_dim,)),
        layers.Dense(output_dim)
    ])
    return model


input_dim = 5
hidden_dim = 10
output_dim = 2

model = build_keras_model(input_dim, hidden_dim, output_dim)
model.compile(optimizer='adam', loss='mse')

x = np.random.randn(3, input_dim)
y = model(x)
print("Выход модели Keras:")
print(y)

### PyTorch
**PyTorch** – это библиотека, разработанная Facebook (Meta, запрещена в РФ), с интуитивно понятной архитектурой и динамическим вычислительным графом.

Особенности:
- Каждый шаг вычислений производится "на лету", что позволяет легко изменять модель.
- Проста в освоении и хорошо интегрирована с Python.
- Наиболее широко используется в академических исследованиях, что способствует быстрому внедрению новых идей.

In [ ]:
import torch
import torch.nn as nn

class SimplePyTorchModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimplePyTorchModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_dim = 5
hidden_dim = 10
output_dim = 2
model = SimplePyTorchModel(input_dim, hidden_dim, output_dim)

x = torch.randn(3, input_dim)
y = model(x)
print("Выход модели PyTorch:")
print(y)